# Importing libraries

In [ ]:
import os
__print__ = print
def print(string):
    os.system(f'echo \"{string}\"')
    __print__(string)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Dropout
from keras.models import Model

import os,cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pylab import rcParams
rcParams['figure.figsize'] = 20, 10

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import pandas as pd# Any results you write to the current directory are saved as output.
from IPython.display import display, Image
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout, BatchNormalization,Activation,UpSampling2D,Add

import tensorflow as tf
# Any results you write to the current directory are saved as output.
from IPython.display import display, Image

In [ ]:
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

# Loading Dataset

In [ ]:
class JoinedGen(tf.keras.utils.Sequence):
    def __init__(self, input_gen, target_gen):
        self.input_gen = input_gen
        self.target_gen = target_gen

        assert len(input_gen) == len(target_gen)

    def __len__(self):
        return len(self.input_gen)

    def __getitem__(self, i):
        x = self.input_gen[i]
        y = self.target_gen[i]

        return x, y

    def on_epoch_end(self):
        self.input_gen.on_epoch_end()
        self.target_gen.on_epoch_end()
        self.target_gen.index_array = self.input_gen.index_array

        
def get_generator(df,shuffle,rescale,batchSize):
    generator_imgs = ImageDataGenerator(rescale = rescale)
    
    clean_img_gen = generator_imgs.flow_from_dataframe(dataframe=df,
                                           x_col="clean", 
                                           class_mode=None,
                                           target_size=(224,224),
                                           validate_filenames = True,
                                           shuffle = shuffle,
                                           batch_size=batchSize)
    
    adv_img_gen = generator_imgs.flow_from_dataframe(dataframe=df,
                                           x_col="adv", 
                                           class_mode=None,
                                           target_size=(224,224),
                                           validate_filenames = True,
                                           shuffle = shuffle,
                                           batch_size=batchSize)
    
    return JoinedGen(adv_img_gen,clean_img_gen)



In [ ]:
def get_filenames(text_file, clean_directory ,adv_directory):
    
    clean_imgs = list()
    adv_imgs = list()
    
    with open(text_file,'r') as f:
        for line in f.readlines():
            line = line.split()
            clean_filename = clean_directory + line[0]
            adv_filename = adv_directory  + line[1]
            clean_imgs.append(clean_filename)
            adv_imgs.append(adv_filename)
            
    return clean_imgs,adv_imgs

In [ ]:
adv_image_dir = '../input/covidx-adversarial-samples/hgd_adv/content/drive/MyDrive/Capstone Project/Dataset/data/forHGD_new/'
clean_image_dir = '../input/covidx-adversarial-samples/hgd_clean/content/drive/MyDrive/Capstone Project/Dataset/data/forHGD_new_clean/'

train_metadata = '../input/covidx-adversarial-samples/forHGD_new_train.txt'
val_metadata = '../input/covidx-adversarial-samples/forHGD_new_val.txt'
test_metadata = '../input/covidx-adversarial-samples/forHGD_new_test.txt'

train_clean,train_adv = get_filenames(train_metadata,clean_image_dir,adv_image_dir)
val_clean,val_adv = get_filenames(val_metadata,clean_image_dir,adv_image_dir)
test_clean,test_adv = get_filenames(test_metadata,clean_image_dir,adv_image_dir)


df_train = pd.DataFrame({'clean' : train_clean, 'adv' : train_adv})
df_test = pd.DataFrame({'clean' : test_clean, 'adv' : test_adv})
df_val = pd.DataFrame({'clean' : val_clean , 'adv' : val_adv})


In [ ]:
train_gen = get_generator(df_train,shuffle = True, rescale = 1/255 , batchSize = 24)
valid_gen = get_generator(df_val,shuffle = True, rescale = 1/255 , batchSize = 24)
test_gen = get_generator(df_test,shuffle = True, rescale = 1/255, batchSize =24 )

# Construction of Model

## *Feedforward Model*

In [ ]:
"""
input_layer = Input(shape=(224,224, 3))


# Feed Forward path

#Layer-1  C2
layer1_conv1 = Conv2D(64,3,padding = 'same')(input_layer)
layer1_conv1  = BatchNormalization()(layer1_conv1)
layer1_conv1 = Activation("relu")(layer1_conv1)
layer1_conv2 = Conv2D(64,3,padding = 'same')(layer1_conv1)
layer1_conv2  = BatchNormalization()(layer1_conv2)
layer1_conv2 = Activation("relu")(layer1_conv2)


#Layer-2 C3

layer2_conv1 = Conv2D(128,3,strides = (2,2),padding = 'same')(layer1_conv2)
layer2_conv1  = BatchNormalization()(layer2_conv1)
layer2_conv1 = Activation("relu")(layer2_conv1)

layer2_conv2 = Conv2D(128,3,padding = 'same')(layer2_conv1)
layer2_conv2  = BatchNormalization()(layer2_conv2)
layer2_conv2 = Activation("relu")(layer2_conv2)

layer2_conv3 = Conv2D(128,3,padding = 'same')(layer2_conv2)
layer2_conv3  = BatchNormalization()(layer2_conv3)
layer2_conv3 = Activation("relu")(layer2_conv3)

#Layer-3 C3



layer3_conv1 = Conv2D(256,3,strides = (2,2),padding = 'same')(layer2_conv3)
layer3_conv1  = BatchNormalization()(layer3_conv1)
layer3_conv1 = Activation("relu")(layer3_conv1)

layer3_conv2 = Conv2D(256,3,padding = 'same')(layer3_conv1)
layer3_conv2  = BatchNormalization()(layer3_conv2)
layer3_conv2 = Activation("relu")(layer3_conv2)

layer3_conv3 = Conv2D(256,3,padding = 'same')(layer3_conv2)
layer3_conv3  = BatchNormalization()(layer3_conv3)
layer3_conv3 = Activation("relu")(layer3_conv3)


#Layer-4 C3


layer4_conv1 = Conv2D(256,3,strides =(2,2),padding = 'same')(layer3_conv3)
layer4_conv1  = BatchNormalization()(layer4_conv1)
layer4_conv1 = Activation("relu")(layer4_conv1)

layer4_conv2 = Conv2D(256,3,padding = 'same')(layer4_conv1)
layer4_conv2  = BatchNormalization()(layer4_conv2)
layer4_conv2 = Activation("relu")(layer4_conv2)

layer4_conv3 = Conv2D(256,3,padding = 'same')(layer4_conv2)
layer4_conv3  = BatchNormalization()(layer4_conv3)
layer4_conv3 = Activation("relu")(layer4_conv3)


#Layer-5 C3

layer5_conv1 = Conv2D(256,3,strides = (2,2),padding = 'same')(layer4_conv3)
layer5_conv1  = BatchNormalization()(layer5_conv1)
layer5_conv1 = Activation("relu")(layer5_conv1)

layer5_conv2 = Conv2D(256,3,padding = 'same')(layer5_conv1)
layer5_conv2  = BatchNormalization()(layer5_conv2)
layer5_conv2 = Activation("relu")(layer5_conv2)

layer5_conv3 = Conv2D(256,3,padding = 'same')(layer5_conv2)
layer5_conv3  = BatchNormalization()(layer5_conv3)
layer5_conv3 = Activation("relu")(layer5_conv3)
"""

## *Feedback Path*

In [ ]:
"""
# Feedback Path

# Layer-6

# F
layer6_bi_interpolation = UpSampling2D(interpolation = 'bilinear')(layer5_conv3)
layer6_fusion = concatenate([layer4_conv3, layer6_bi_interpolation])

# C3
layer6_conv1 = Conv2D(256,3,padding = 'same')(layer6_fusion)
layer6_conv1  = BatchNormalization()(layer6_conv1)
layer6_conv1 = Activation("relu")(layer6_conv1)

layer6_conv2 = Conv2D(256,3,padding = 'same')(layer6_conv1)
layer6_conv2  = BatchNormalization()(layer6_conv2)
layer6_conv2 = Activation("relu")(layer6_conv2)

layer6_conv3 = Conv2D(256,3,padding = 'same')(layer6_conv2)
layer6_conv3  = BatchNormalization()(layer6_conv3)
layer6_conv3 = Activation("relu")(layer6_conv3)

#Layer -7 

# F
layer7_bi_interpolation = UpSampling2D(interpolation = 'bilinear')(layer6_conv3)
layer7_fusion = concatenate([layer3_conv3, layer7_bi_interpolation])

# C3
layer7_conv1 = Conv2D(256,3,padding = 'same')(layer7_fusion)
layer7_conv1  = BatchNormalization()(layer7_conv1)
layer7_conv1 = Activation("relu")(layer7_conv1)

layer7_conv2 = Conv2D(256,3,padding = 'same')(layer7_conv1)
layer7_conv2  = BatchNormalization()(layer7_conv2)
layer7_conv2 = Activation("relu")(layer7_conv2)

layer7_conv3 = Conv2D(256,3,padding = 'same')(layer7_conv2)
layer7_conv3  = BatchNormalization()(layer7_conv3)
layer7_conv3 = Activation("relu")(layer7_conv3)



#Layer -8 

# F
layer8_bi_interpolation = UpSampling2D(interpolation = 'bilinear')(layer7_conv3)
layer8_fusion = concatenate([layer2_conv3, layer8_bi_interpolation])

# C3
layer8_conv1 = Conv2D(128,3,padding = 'same')(layer8_fusion)
layer8_conv1  = BatchNormalization()(layer8_conv1)
layer8_conv1 = Activation("relu")(layer8_conv1)

layer8_conv2 = Conv2D(128,3,padding = 'same')(layer8_conv1)
layer8_conv2  = BatchNormalization()(layer8_conv2)
layer8_conv2 = Activation("relu")(layer8_conv2)

layer8_conv3 = Conv2D(128,3,padding = 'same')(layer8_conv2)
layer8_conv3  = BatchNormalization()(layer8_conv3)
layer8_conv3 = Activation("relu")(layer8_conv3)

#Layer -9 

# F
layer9_bi_interpolation = UpSampling2D(interpolation = 'bilinear')(layer8_conv3)
layer9_fusion = concatenate([layer1_conv2, layer9_bi_interpolation])

# C2
layer9_conv1 = Conv2D(64,3,padding = 'same')(layer9_fusion)
layer9_conv1  = BatchNormalization()(layer9_conv1)
layer9_conv1 = Activation("relu")(layer9_conv1)

layer9_conv2 = Conv2D(64,3,padding = 'same')(layer9_conv1)
layer9_conv2  = BatchNormalization()(layer9_conv2)
layer9_conv2 = Activation("relu")(layer9_conv2)

# Layer-10 

# Conv 1x1

layer_10_conv1 = Conv2D(3,1,padding='same')(layer9_conv2)

# Layer-11

layer_11_sum = Add()([layer_10_conv1,input_layer])

output_layer = layer_11_sum
"""

In [ ]:
# model_unet = Model(input_layer,output_layer)
# model_unet.summary()

In [ ]:
# import tensorflow as tf
# tf.keras.utils.plot_model(model_unet, show_shapes=True)

# *Training Of Model*


In [ ]:
# model_unet.compile(optimizer='adam', loss='MSE',metrics=["accuracy"])

In [ ]:
# !mkdir './Model_19March/'

In [ ]:

# import keras.callbacks
# path = './Model_19March/'
# class CustomSaver(keras.callbacks.Callback):
#     def on_epoch_end(self, epoch, logs={}):  # or save after some epoch, each k-th epoch etc.
#             self.model.save(path + "/model_{}".format(epoch))
# saver = CustomSaver()

In [ ]:
# with tf.device('/gpu:0'):
#     history = model_unet.fit(train_gen,
#                     epochs=100,
#                     shuffle=True,
#                     validation_data=valid_gen,
#                     callbacks=[saver])

In [ ]:
# import cv2
# import time
# test_gen.input_gen.reset()
# test_gen.target_gen.reset()

# adv = model_unet.predict(test_gen)
# clean = list()
# fname = list()
# for i in range(len(df_test)):
#     img = cv2.imread(df_test['clean'][i])
#     img = cv2.normalize(img, None, alpha = 0, beta = 1, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
#     fname.append(df_test['clean'][i])
#     print(i)
#     clean.append(img)

In [ ]:
# from skimage.measure import compare_ssim, compare_psnr
# from skimage.metrics import structural_similarity 
# from skimage import data, img_as_float

# print(structural_similarity(np.array(clean), adv,multichannel = True))

# Construction of Model

In [ ]:
try:
    del model_unet
except:
    pass

In [ ]:
def build_model(input_layer):
    conv1 = Conv2D(64,(3,3), activation='relu', padding='same')(input_layer)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(64,(3,3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D((2,2))(conv1)
#     pool1 = Dropout(0.25)(pool1)
    
#     conv2 = Conv2D(128,(3,3),strides = (2,2), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128,(3,3), activation='relu', padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(128,(3,3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(128,(3,3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D((2,2))(conv2)
#     pool2 = Dropout(0.5)(pool2)

#     conv3 = Conv2D(256,(3,3),strides =(2,2), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256,(3,3), activation='relu', padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(256,(3,3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(256,(3,3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D((2,2))(conv3)
#     pool3 = Dropout(0.5)(pool3)
    
#     conv4 = Conv2D(256,(3,3),strides =(2,2), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256,(3,3), activation='relu', padding='same')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(256,(3,3), activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(256,(3,3), activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    pool4 = MaxPooling2D((2,2))(conv4)
#     pool4 = Dropout(0.5)(pool4)

#     convm = Conv2D(256,(3,3),strides =(2,2), activation='relu', padding='same')(pool4)
    convm = Conv2D(256,(3,3), activation='relu', padding='same')(pool4)
    convm = BatchNormalization()(convm)
    convm = Conv2D(256,(3,3), activation='relu', padding='same')(convm)
    convm = BatchNormalization()(convm)
    convm = Conv2D(256,(3,3), activation='relu', padding='same')(convm)
    convm = BatchNormalization()(convm)
    
    deconv4 = Conv2DTranspose(256,(3,3), strides=(2,2), padding='same')(convm)
    deconv4 = BatchNormalization()(deconv4)
    
    uconv4 = concatenate([deconv4, conv4])
    uconv4 = Dropout(0.5)(uconv4)
    uconv4 = Conv2D(256,(3,3), activation='relu', padding='same')(uconv4)
    uconv4 = BatchNormalization()(uconv4)
    uconv4 = Conv2D(256,(3,3), activation='relu', padding='same')(uconv4)
    uconv4 = BatchNormalization()(uconv4)
    uconv4 = Conv2D(256,(3,3), activation='relu', padding='same')(uconv4)
    uconv4 = BatchNormalization()(uconv4)
    
    deconv3 = Conv2DTranspose(256,(3,3), strides=(2,2), padding='same')(uconv4)
    deconv3 = BatchNormalization()(deconv3)

    uconv3 = concatenate([deconv3, conv3])
#     uconv3 = Dropout(0.5)(uconv3)
    uconv3 = Conv2D(256, (3,3), activation='relu', padding='same')(uconv3)
    uconv3 = BatchNormalization()(uconv3)
    uconv3 = Conv2D(256, (3,3), activation='relu', padding='same')(uconv3)
    uconv3 = BatchNormalization()(uconv3)
    uconv3 = Conv2D(256, (3,3), activation='relu', padding='same')(uconv3)
    uconv3 = BatchNormalization()(uconv3)
    
    deconv2 = Conv2DTranspose(128,(3,3), strides=(2,2), padding='same')(uconv3)
    deconv2 = BatchNormalization()(deconv2)
    
    uconv2 = concatenate([deconv2, conv2])
#     uconv2 = Dropout(0.5)(uconv2)
    uconv2 = Conv2D(128, (3,3), activation='relu', padding='same')(uconv2)
    uconv2 = BatchNormalization()(uconv2)
    uconv2 = Conv2D(128, (3,3), activation='relu', padding='same')(uconv2)
    uconv2 = BatchNormalization()(uconv2)
    uconv2 = Conv2D(128, (3,3), activation='relu', padding='same')(uconv2)
    uconv2 = BatchNormalization()(uconv2)

    
    deconv1 = Conv2DTranspose(64,(3,3), strides=(2,2), padding='same')(uconv2)
    deconv1 = BatchNormalization()(deconv1)
    uconv1 = concatenate([deconv1, conv1])
#     uconv1 = Dropout(0.5)(uconv1)
    uconv1 = Conv2D(64, (3,3), activation='relu', padding='same')(uconv1)
    uconv1 = BatchNormalization()(uconv1)
    uconv1 = Conv2D(64, (3,3), activation='relu', padding='same')(uconv1)
    uconv1 = BatchNormalization()(uconv1)
    
    last_layer = Conv2D(3, (1,1), padding='same', activation = 'sigmoid')(uconv1)
    
    
    output_layer = Add()([last_layer,input_layer])
    return output_layer
    

In [ ]:
# def build_model(input_layer, start_neurons):
#     64
#     conv1 = Conv2D(start_neurons*1,(3,3), activation='relu', padding='same')(input_layer)
#     conv1 = Conv2D(start_neurons*1,(3,3), activation='relu', padding='same')(conv1)
#     pool1 = MaxPooling2D((2,2))(conv1)
#     pool1 = Dropout(0.25)(pool1)
    
#     conv2 = Conv2D(start_neurons*2,(3,3), activation='relu', padding='same')(pool1)
#     conv2 = Conv2D(start_neurons*2,(3,3), activation='relu', padding='same')(conv2)
#     pool2 = MaxPooling2D((2,2))(conv2)
#     pool2 = Dropout(0.5)(pool2)

#     conv3 = Conv2D(start_neurons*4,(3,3), activation='relu', padding='same')(pool2)
#     conv3 = Conv2D(start_neurons*4,(3,3), activation='relu', padding='same')(conv3)
#     pool3 = MaxPooling2D((2,2))(conv3)
#     pool3 = Dropout(0.5)(pool3)
    
#     conv4 = Conv2D(start_neurons*8,(3,3), activation='relu', padding='same')(pool3)
#     conv4 = Conv2D(start_neurons*8,(3,3), activation='relu', padding='same')(conv4)
#     pool4 = MaxPooling2D((2,2))(conv4)
#     pool4 = Dropout(0.5)(pool4)

#     #Middle
#     convm = Conv2D(start_neurons * 16, (3,3), activation='relu', padding='same')(pool4)
#     convm = Conv2D(start_neurons * 16, (3,3), activation='relu', padding='same')(convm)
    
#     #upconv part
#     deconv4 = Conv2DTranspose(start_neurons*8,(3,3), strides=(2,2), padding='same')(convm)
#     uconv4 = concatenate([deconv4, conv4])
#     uconv4 = Dropout(0.5)(uconv4)
#     uconv4 = Conv2D(start_neurons*8, (3,3), activation='relu', padding='same')(uconv4)
#     uconv4 = Conv2D(start_neurons*8, (3,3), activation='relu', padding='same')(uconv4)
    
#     deconv3 = Conv2DTranspose(start_neurons*8,(3,3), strides=(2,2), padding='same')(uconv4)
#     uconv3 = concatenate([deconv3, conv3])
#     uconv3 = Dropout(0.5)(uconv3)
#     uconv3 = Conv2D(start_neurons*4, (3,3), activation='relu', padding='same')(uconv3)
#     uconv3 = Conv2D(start_neurons*4, (3,3), activation='relu', padding='same')(uconv3)
    
#     deconv2 = Conv2DTranspose(start_neurons*8,(3,3), strides=(2,2), padding='same')(uconv3)
#     uconv2 = concatenate([deconv2, conv2])
#     uconv2 = Dropout(0.5)(uconv2)
#     uconv2 = Conv2D(start_neurons*2, (3,3), activation='relu', padding='same')(uconv2)
#     uconv2 = Conv2D(start_neurons*2, (3,3), activation='relu', padding='same')(uconv2)
    
#     deconv1 = Conv2DTranspose(start_neurons*8,(3,3), strides=(2,2), padding='same')(uconv2)
#     uconv1 = concatenate([deconv1, conv1])
#     uconv1 = Dropout(0.5)(uconv1)
#     uconv1 = Conv2D(start_neurons*1, (3,3), activation='relu', padding='same')(uconv1)
#     uconv1 = Conv2D(start_neurons*1, (3,3), activation='relu', padding='same')(uconv1)
    
#     output_layer = Conv2D(3, (1,1), padding='same', activation='sigmoid')(uconv1)
#     return output_layer

In [ ]:
#Defining target image size and Input size and building model
img_size_target = 224
input_layer = Input((img_size_target, img_size_target,3))
output_layer = build_model(input_layer)

In [ ]:
from keras.models import load_model
surrogate_model = load_model('../input/best-model/Best_Model_20.h5')

In [ ]:
def loss_func(y_true,y_pred):
    true = surrogate_model(y_true)
    pred = surrogate_model(y_pred)
#     mse_loss = tf.keras.losses.MSE(y_true,y_pred)
    hgd_loss = tf.math.reduce_mean(tf.abs(true - pred))
#     pgd_loss = tf.math.reduce_mean(tf.abs(y_true - y_pred))
    # msle
#     return tf.add(hgd_loss,pgd_loss)
    return hgd_loss

In [ ]:
#Initializing and compiling model
model_unet = Model(input_layer, output_layer)
model_unet.compile(optimizer='adam', loss=loss_func)

# UNET Summary

In [ ]:
# model_unet.summary()

# Training Model

In [ ]:
import keras

checkpoint_path = "Model_PGD"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)


In [ ]:
# model_unet.fit(x_train_noisy, x_train,
#                 epochs=15,
#                 batch_size=64,
#                 shuffle=True,
#                 validation_data=(x_test_noisy, x_test))
# import keras.callbacks
# path = './HGD_MODEL_Kaggle/'
# class CustomSaver(keras.callbacks.Callback):
#     def on_epoch_end(self, epoch, logs={}):  # or save after some epoch, each k-th epoch etc.
#             self.model.save(path + "model_{}".format(epoch))
# saver = CustomSaver()

with tf.device('/device:GPU:0'):
    history = model_unet.fit(train_gen,
                    epochs=100,
                    shuffle=True,
                    validation_data=valid_gen,
                    callbacks = [cp_callback])

In [ ]:
from matplotlib import pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()